In [ ]:
from __future__ import annotations
import re
from functools import reduce


"""event class module

This module contains the class event, which represents a single event
in a timeline. It containes the timeline, the event, and the start and
end time of the event.

Allows for easy comparison and hashing of events as well as printing.

Example:
    >>> from event import event
    >>> e = eventClass(1, 'a', 0, 1)
    >>> print(e)
    (1, a, (0, 1))


"""


class eventClass():
    """Class representing a single event in a timeline.

    This class represents a single event in a timeline. It containes the
    timeline, the event, and the start and end time of the event.

    Attributes:
        timeline (int): The timeline of the event.
        event (str): The event.
        points (tuple): The start and end time of the event.
        
    Raises:
        ValueError: If the event is not in the correct format.

    """

    def __init__(self, timeline: int, event: str, points: tuple[int, int]):
        if not eventClass.check_format((timeline, event, points)):
            raise ValueError("Invalid event format")
        
        self.timeline = timeline
        self.event = event
        self.start = points[0]
        self.end = points[1]

    def __repr__(self):
        return f"event({self.timeline}, {self.event}, {self.start}, {self.end})"

    def __str__(self):
        return f"({self.timeline}, {self.event}, ({self.start}, {self.end}))"

    def __eq__(self, other):
        return self.timeline == other.timeline and self.event == other.event and self.start == other.start and self.end == other.end

    def __hash__(self):
        return hash((self.timeline, self.event, self.start, self.end))
    
    def __getitem__(self, key):
        if key == 0 or key == -1:
            return self.timeline
        elif key == 1 or key == -2:
            return self.event
        elif key == 2 or key == -3:
            return (self.start, self.end)
        else:
            raise IndexError("Event has only 3 elements")


    def check_format(obj):
        """Check if an object is a valid event.

        Runs all the checks to see if an object is a valid event.
        Returns True if it is, False otherwise.

        Args:
            obj (object): The object to check.

        Returns:
            bool: True if the object is a valid event, False otherwise.

        """

        if isinstance(obj, eventClass):
            return True

        return (
            # Tuple or list, no dict or anything else
            isinstance(obj, tuple) and not isinstance(obj, list)
            and len(obj) == 3  # 3 elements, timeline, event and start and end
            and isinstance(obj[0], int)  # Timeline is an int
            and isinstance(obj[1], str)  # event name is a string
            # Event[2] is a tuple or a list
            and (isinstance(obj[2], tuple) or isinstance(obj[2], list))
            and len(obj[2]) == 2  # Event[2] has 2 elements, start and end
            and isinstance(obj[2][0], int)  # Start is an int
            and isinstance(obj[2][1], int)  # End is an int
            and obj[2][0] < obj[2][1]  # Start is before end
        )


"""Base class for lexical representations of events.

This module contains the base class for lexical representations of events. 
It provides basic interface and wrappers to handle events in the form of 
lists of lists of strings. This class provides no functionality beside 
transforming data, both inputted and outputted, into the correct format.

Example:
    The following example shows how to use the base class to create a
    lexical representation of events:
        
        >>> from lex_base import baseLexRepr
        >>> data = [['a', 'b', 'c'], ['d', 'e', 'f']]
        >>> lex = baseLexRepr(data)
        >>> print(lex)
        a b c
        d e f


"""





class baseLexRepr():
    """Base class for lexical representations of events.

    This class provides basic interface and wrappers to handle events in the form of
    lists of lists of strings. This class provides no functionality beside transforming
    data, both inputted and outputted, into the correct format.

    Attributes:
        data: The data to be wrapped into a lexical representation.

    Raises:
        ValueError: If the input data has the wrong format.

    """

    def __init__(self, input: list[list[str]]):
        # Check if the input is acceptable
        if not self.check_format(input):
            raise ValueError(f"Input data has wrong format. Got {input}")

        self.data = input

        self.timelines = len(self.data[0])

        self._as_regex = None
        self._as_searchable_string = None
        self._event_list = None

    ### Methods for handling attributes ###

    # Regex representation
    @property
    def as_regex(self) -> str:
        """str: The lexical representation of the data as a regular expression.

        This representation is used to determine if an istance is contained into another.
        Lazily computed singleton. Will need to be re-computed if the data changes."""

        if self._as_regex is None:
            self._as_regex = self._to_re()
        return self._as_regex

    @as_regex.deleter
    def as_regex(self) -> None:
        self._as_regex = None

    def _to_re(self) -> str:
        """str: The lexical representation of the data as a regular expression.

        This function processes the dataset and outputs a regular expression that can be used to match it
        inside another dataset.
        The regular expression is generated by replacing the events with their
        literal representation and adding fillers between them to add flexibility in recognition.
        The fillers are represented by a list of any event that can be matched 0 or more times.

        Returns:
            str: The lexical representation of the data as a regular expression.
        """

        # TODO: Change this to avoid needing a copy

        # Copy to avoid modifying the original
        data = self.copy()

        # Insert fillers between events to add flexibility in recognition
        for i in range(1, len(data.data)):
            data.data.insert(i * 2 - 1, data.gen_null(i * 2 - 1))

        # Convert to regex
        for row in range(len(data)):
            for column in range(len(data[row])):
                value = data[row][column]

                # If the value is blank we recognize any event
                if value == "_":
                    data[row][column] = r"[a-zA-Z0-9_-]+"

                # If the value is an endind event we recognize either an end or another start
                elif value.startswith("E"):
                    data[row][column] = r"([SE]_[a-zA-Z0-9_-]+)"

                # Every other event is recognized as a literal of itself
                else:
                    data[row][column] = f"{value}"

        # Generate regex. Even positions contains filler that can be matched 0 or more times, odd positions contains events that must be matched exactly
        regex = ""
        for it, i in enumerate(data.data):
            if it % 2 == 0:
                # chr(92) is the backslash character. The f-string wouldn't allow to use it directly
                regex += f'(\[{(chr(92)+"s*,"+chr(92)+"s*").join(i)}\])'
            else:
                regex += f'(?:\[{(chr(92)+"s*,"+chr(92)+"s*").join(i)}\])*?'

        return regex

    # Searchable string representation
    @property
    def as_searchable_string(self) -> str:
        """str: The lexical representation of the data as a searchable string.

        Searchable because it is in a format compatible with regular expressions.
        This representation is used to determine if an istance is contained into another.
        Lazily computed singleton. Will need to be re-computed if the data changes."""

        if self._as_searchable_string is None:
            self._as_searchable_string = self._to_searchable_string()
        return self._as_searchable_string

    def _to_searchable_string(self) -> str:
        return "".join(f"[{j}]" for j in [",".join(i) for i in self.data])

    @as_searchable_string.deleter
    def as_searchable_string(self):
        self._as_searchable_string = None

    # Event list
    @property
    def events_list(self) -> list[eventClass]:
        """list of strings: The list of events contained in the dataset.

        This is useful to iterate over the events in the dataset.
        It is also used for determining the dataset size and number of events.
        Lazily computed singleton. Will need to be re-computed if the data changes."""

        if self._event_list is None:
            self._event_list = self._get_events()
        return self._event_list

    @events_list.deleter
    def events_list(self) -> None:
        self._event_list = None

    def _get_events(self) -> list[eventClass]:
        # TODO: Change this behavior to output a list of event objects
        # TODO: avoid usage of support functions from preprocess

        events = {}

        # Empty check
        if len(self.data) == 0:
            return events

        # Examine timeline by timeline
        for timeline in range(len(self.data[0])):
            events[timeline] = []

            # Every instant on a timeline
            for moment in range(len(self.data)):
                # If a Start event is found, we have to find the corresponding End event
                if self.data[moment][timeline].startswith("S"):
                    event = self.data[moment][timeline].split("_")[1]
                    start_time = moment
                    end_time = moment + 1

                    # Go look for the next End (or Start) event
                    while end_time < len(self.data):
                        if self.data[end_time][timeline].startswith("S"):
                            break
                        if self.data[end_time][timeline].startswith("E"):
                            break
                        end_time += 1

                    # Save the event
                    events[timeline].append((event, (start_time, end_time)))


        # Transform the dictionary
        # {timeline: [(event, start_time, end_time), ...], ...}
        # into a list of events
        # [(timeline, (timeline, event, start_time, end_time)), ...]
        return [eventClass(n_timeline, *event) for n_timeline in events for event in events[n_timeline]]

    ### Methods for checking validity ###
    @staticmethod
    def check_format(input: list[list[str]]) -> bool:
        """Test if the input data is valid and can be used to create a lexical representation of events.

        Not acceptable formats include:
            - empty data
            - data with empty timelines
            - data with timelines of different lengths
            - data with non-string events

        Moreover, the data must present sequence of events, i.e. there must be at least one Start event,
        and the Start and End events must be correctly paired. if an event starts, it must end.
        No holes are allowed in the data: no empty strings, between start and end events intermediates are required.

            Args:
                input: The data to be checked.

            Returns:
                True if the data is valid, False otherwise.

        """

        # Check that the input is a list of lists and nonempty
        if len(input) == 0 or not isinstance(input, list):
            return False
        if len(input[0]) == 0 or not isinstance(input[0], list):
            return False

        # Check that all timelines have the same length
        for i in input:
            if len(i) != len(input[0]):
                return False

        # Check that all events are strings and input shape
        for i in input:
            for j in i:
                if not isinstance(j, str):
                    
                    return False

        return baseLexRepr.check_well_formed(input)

    @staticmethod
    def check_well_formed(test_data: list[list[str]]) -> bool:
        """Test if the input data is well-formed.

        This function tests if the input data is well-formed.
        A well-formed dataset is one that has no empty events, and where every Start event is followed by an End event.
        Moreover, between a Start and an End event there are only intermediate events.
        Also, no empty value is allowed. Only S_, E_, I_ and _ are allowed.

        Args:
            test_data: The data to be checked.

        Returns:
            True if the data is well-formed, False otherwise.

        """

        # Check that the representation is well-formed:
        # - No empty events
        # - Every End is preceeded by a Start
        # - Every Start is followed by an End or another Start
        # - Between a Start and an End (or another Start) there are only intermediate events

        for column in range(len(test_data)):
            # Check if the column contains only valid values and are splittable in 2 parts
            if not all(
                row[:2] in ["S_", "E_", "I_", "_"]
                and (len(row.split("_")) == 2 or row == "_")
                for row in test_data[column]
            ):
                return False

            if all([i == "_" for i in test_data[column]]):
                return False

            # Check all instants
            for row in range(len(test_data[column])):
                # If it's the starting instant
                if column == 0:
                    # It cannot be other than a Start or a blank event
                    if test_data[column][row].startswith("I") or test_data[column][
                        row
                    ].startswith("E"):
                        print(test_data[column][row])
                        return False

                # If it's the ending instant
                elif column == len(test_data) - 1:
                    # It cannot be other than an End or a blank event
                    if test_data[column][row].startswith("I") or test_data[column][
                        row
                    ].startswith("S"):
                        print(test_data[column][row])
                        return False

                # For all other instants
                else:
                    # If it's an intermediate event
                    if test_data[column][row].startswith("I"):
                        # Check left neighbor
                        # Only Start or Intermediate are allowed and if they are, they must be of the same event
                        if not (
                            (
                                test_data[column - 1][row].startswith("S")
                                or test_data[column - 1][row].startswith("I")
                            )
                            and test_data[column - 1][row].split("_")[1]
                            == test_data[column][row].split("_")[1]
                        ):
                            print(test_data[column][row])
                            return False

                        # Check right neighbor
                        # Only Start, End or Intermediate are allowed and if are Intermediate or End, they must be of the same event
                        if not (
                            test_data[column + 1][row].startswith("S") or
                            (
                                test_data[column + 1][row].startswith("E")
                                or test_data[column + 1][row].startswith("I")
                            )
                            and test_data[column + 1][row].split("_")[1]
                            == test_data[column][row].split("_")[1]
                        ):
                            print(test_data[column][row])
                            return False

                    # If it's an End event
                    if test_data[column][row].startswith("E"):
                        # Check left neighbor
                        # Only Start or Intermediate are allowed and if they are, they must be of the same event
                        if not (
                            (
                                test_data[column - 1][row].startswith("S")
                                or test_data[column - 1][row].startswith("I")
                            )
                            and test_data[column - 1][row].split("_")[1]
                            == test_data[column][row].split("_")[1]
                        ):
                            print(test_data[column][row])
                            return False

                        # Check right neighbor
                        # Only another Start or an empty event are allowed
                        if not (
                            test_data[column + 1][row].startswith("_")
                            or test_data[column + 1][row].startswith("S")
                        ):
                            print(test_data[column][row])
                            return False

                    if test_data[column][row].startswith("S"):
                        # Left neighbor may be anything
                        
                        # Check right neighbor
                        # A blank event is not allowed and if it is an End or Intermediate event, it must be of the same event
                        if not (
                            not test_data[column + 1][row].startswith("_")
                            or (
                                (
                                    test_data[column + 1][row].startswith("E")
                                    or test_data[column + 1][row].startswith("I")
                                )
                                and test_data[column + 1][row].split("_")[1]
                                == test_data[column][row].split("_")[1]
                            )
                        ):
                            print(test_data[column][row])
                            return False
        return True

    @staticmethod
    def from_event(event: tuple | list | eventClass, total_timelines: int = 1) -> list:
        """Create a lexical representation of a single event.
        
        This function creates a lexical representation of a single event.
        The event must be in the form (timeline, label, (start_index, end_index)) or lib.event.eventClass.
        This is useful for creating a lexical representation of a single event compatible with another itemset.
        
        Args:
            event: The event to be represented.          
            total_timelines: The total number of timelines in the dataset.
            
        Returns:
            A list containing the lexical representation of the event.
            
        Raises:
            ValueError: If the event has the wrong shape.
                    
        """

        # Check if the input is a well-formatted event object
        if not isinstance(event, eventClass):
            if not eventClass.check_format(event):
                raise ValueError(f"Input event has wrong format. Got {event}")
            else:
                event = eventClass(*event)

        # Create the object
        data = [['_' for _ in range(total_timelines)] for _ in range(1, 2+1)]
        data[0][event.timeline] = f'S_{event.event}'
        data[1][event.timeline] = f'E_{event.event}'

        return data

    def check_compatibility(self, input) -> bool:
        """Check if the input is compatible with the current object.

        This function checks if the input object is compatible with the current object.
        Two objects are compatible if they have the same number of timelines.

        Args:
            input: The object to be checked.

        Returns:
            True if the object is compatible, False otherwise.

        """

        if not isinstance(input, baseLexRepr):
            return False

        return self.timelines == input.timelines

    ### Methods for interacting ###

    # Size of the dataset (number of instants)
    def __len__(self) -> int:
        """int: The number of instants in the dataset."""

        return len(self.data)

    # Size of the dataset (number of events)
    @property
    def size(self) -> int:
        """int: The number of events in the dataset."""

        return len(self.events_list)

    # String representation
    def __str__(self) -> str:
        output = ""

        for j in range(len(self.data[0])):
            for i in self.data:
                if i[j] == "_":
                    output += f" \u00d8\t"
                else:
                    output += f"{i[j]}\t"
            output += "\n"

        return output

    # Representation
    def __repr__(self) -> str:
        return f"baseLexRepr({self.data})"

    # Containment
    def __contains__(self, query: eventClass | baseLexRepr) -> bool:
        """Check if the input is contained in the current object.

        The function allows both single events and other lexical representations to be used as input.
        If the input is a lexical representation it gets turned into a regular expression and then matched
        against the searchable string representation of the current object.
        If it is an event object it gets confronted with the events list in the current object.

        Args:
            query: The object to be checked.

        Returns:
            True if the object is contained, False otherwise.

        """
        if isinstance(query, eventClass):
            return query in self.events_list
        else:
            if not (isinstance(query, baseLexRepr) and self.check_compatibility(query)):
                return False

        # Return search result. True if anything has been found, False otherwise
        return re.search(query.as_regex, self.as_searchable_string) is not None

    # Wrap data list
    def __getitem__(self, index) -> list[str]:
        return self.data[index]

    # Here may be necessary to override to invalidate cached values of children classes
    def __delitem__(self, index: int) -> None:
        if index > len(self):
            raise IndexError("Index out of range")
        
        del self.data[index]

        # Data changed, invalidate cached values
        self._as_regex = None
        self._as_searchable_string = None
        self._event_list = None

    def __iter__(self):
        return iter(self.data)

    def __next__(self):
        return next(self.data)

    # Here may be necessary to override to change how equality and hashing work
    def __eq__(self, o: object) -> bool:
        return isinstance(o, baseLexRepr) and self.data == o.data

    def __hash__(self) -> int:
        return hash(hash((j for j in i)) for i in self.data)

    ### Data manipulation ###
    def delete_event(self, event: tuple | eventClass) -> baseLexRepr:
        # TODO: Change this to avoid needing a copy
        """Deletes an event from the dataset.

        This function deletes the input event from the dataset.
        The event must be in the form (timeline, (label, start_index, end_index)).
        The event is deleted by replacing the event with '_' in the dataset.

        Args:
            event (tuple | event): The event to be deleted.

        Returns:
            baseLexRepr: A copy of the object with the event deleted.

        Raises:
            ValueError: If the event has the wrong shape.

        """

        # Check if event has the correct shape
        if not isinstance(event, eventClass):
            if not eventClass.check_format(event):
                raise ValueError(
                    f"Event has wrong shape, expected (timeline, label, (start_index, end_index)), got {event}"
                )
            
        temp = self.copy()

        for i in range(event[2][0], event[2][1] + 1):
            # Check that the Start event is not an End event
            if (
                i > 0
                and i == event[2][0]
                and (
                    temp[i - 1][event[0]].startswith("I")
                    or temp[i - 1][event[0]].startswith("S")
                )
            ):
                temp[i][event[0]] = "E_" + temp[i-1][event[0]].split("_")[1]
                continue
            # Check there is no End event but only a new Start event
            if i == event[2][1] and temp[i][event[0]].startswith("S"):
                break
            temp[i][event[0]] = "_"

        temp.del_null()
        return temp.copy()

    def del_null(self) -> None:
        """Removes all the null events

        This function removes all the null events from the dataset.
        Null events are those that consist entirely of '_' and Intermediate
        events.

        """

        changed = False
        j = 0
        # Parses all the events and removes those that are null
        while j < len(self):
            if all([i == "_" or i.startswith("I") for i in self[j]]):
                del self[j]
                changed = True
            else:
                j += 1

        # Data have changed, invalidate cached values
        if changed:
            self._as_regex = None
            self._as_searchable_string = None
            self._event_list = None

    def gen_null(self, index: int) -> list[list[str]]:
        """Generates a null event at the input index.

        This function generates a null event at the input index.
        A null event is an event that is composed only of '_' and Intermediate
        events.
        This function is used to generate reasonable fillers between events
        and mantain data validity.

        Args:
            index: The index where to insert the null event.

        Returns:
            The null event.

        Raises:
            IndexError: If the specified index is larger than the size of the data.

        """

        # Check and sanitation, this function works on a list of lists
        if index > len(self):
            raise IndexError("Index out of range")

        # Insert a blank event at the position
        data = ["_"] * len(self[0])

        # If the insertion happened before everything, there nothing to propagate
        if index != 0:
            # For all timelines in the data, write the inserted instant depending on values before
            for k in range(len(data)):
                # Only if the previous instant was an event, propagate as Intermediate
                if self[index - 1][k].startswith("S") or self[index - 1][k].startswith(
                    "I"
                ):
                    data[k] = "I_" + self[index - 1][k].split("_")[1]
                # Otherwise leave blank

        return data

    # Here may be necessary to override to change copy behavior of children classes
    def copy(self) -> baseLexRepr:
        return baseLexRepr([i.copy() for i in self.data])

    # To be implemented in children classes
    def merge(self, other) -> list[baseLexRepr]:
        if not self.check_compatibility(other):
            raise ValueError("Incompatible objects")

        raise NotImplementedError
def data_to_intervals(data):

    parsed_data = {}

    # Iterate through each timeline
    for timeline, events in data.items():

        parsed_data[timeline] = []

        # Keep track of time index for each timeline
        last_end = 0

        # Iterate through each event
        for event in events:
            # Ignore empty events
            if event[0] is not None:
                begin = last_end
                end = last_end + event[1]
                parsed_data[timeline].append((event[0].replace(' ', '-').replace("'", ''), begin, end))

            
            # Add duration to time index
            last_end += event[1]

    return parsed_data

def dict_to_list(data_dict):
    l = list(data_dict.items())
    l.sort(key=lambda x: x[0])
    return [(k, v) if v else (k, []) for k, v in l]


def replace_current_event(current_events, parsed_data, i):

    '''
    if len(parsed_data[i]) == 0:
        current_events[i] = ()
        return
    '''
    # Update next event. If there is no next event, set null value (empty tuple)
    if len(parsed_data[i][1]) != 0:
        # Extract next event
        current_events[i] = parsed_data[i][1].pop(0)
    else:
        # Set null event
        current_events[i] = ()

def intervals_to_words(data_intervals):
    
    # Extract next events
    current_events = [()]*len(data_intervals)
    [replace_current_event(current_events, data_intervals, i) for i in range(len(data_intervals))]    
    # Keep track of which events are active (default none)
    timelines_status = {data_intervals[i][0]: '_' for i in range(len(data_intervals))}

    # Check if there are no events
    if current_events == [()]*len(data_intervals):
        return []
    
    # Prepare for the analysis starting from negative T and no instants
    lexical_representation = []
    next_t = -1

    while not all(current_events[i] == () for i in range(len(current_events))):
        
        # Define current instant as empty, editing will be put in place by each timeline
        instant_representation = ['_']*len(data_intervals)

        
        # Get next interesting instant
        next_t = min([j for j in [i[1] for i in current_events if i != ()] + [i[2] for i in current_events if i != ()] if j > next_t])

        # Cycles through each timeline
        for i in range(len(data_intervals)):

            # There is no event, '_' is fine
            if current_events[i] == ():
                continue
            
            # Check if this time instant is a start event
            elif next_t == current_events[i][1]:
                # Place S_x
                instant_representation[i] = f'S_{current_events[i][0]}'
                # Remember that event is active
                timelines_status[data_intervals[i][0]] = current_events[i][0]

                # Check if skip E_ event
                # If the end of this event is the same as the start of the next event
                # Skip this event to replace this E_x with the next S_x
                if len(data_intervals[i][1]) != 0 and current_events[i][2] == data_intervals[i][1][0][1]:
                    replace_current_event(current_events, data_intervals, i)
            
            # Check if this time instant is a start event
            elif next_t == current_events[i][2]:
                # Place E_x
                instant_representation[i] = f'E_{current_events[i][0]}'
                # Remember that event is not active
                timelines_status[data_intervals[i][0]] = '_'
                # Update current event
                replace_current_event(current_events, data_intervals, i)

            # Check if this time instant does not involve this timeline
            else:
                # If timeline has an active event, place I_x
                if timelines_status[data_intervals[i][0]] != '_':
                    instant_representation[i] = f'I_{timelines_status[data_intervals[i][0]]}'
                    
        # Add instant to representation
        lexical_representation.append(instant_representation)
    return lexical_representation

def data_to_words(data):

    # Turn data into intervals (could possibily be skipped)
    data_intervals = dict_to_list(data_to_intervals(data))
    
    return intervals_to_words(data_intervals)

def intervals_as_durations(intervals):
    if len(intervals) == 0:
        return [(None, 1)]
    real_intervals = reduce(lambda x,y: x.union(y), intervals[BE])
    complements = list(Allen.closed(0, max(intervals[E])).difference(real_intervals)) 
    total_intervals = complements + list(real_intervals)
    total_intervals.sort()
    r = [ (str(int(intervals[intervals[BE] == i][TK].iloc[0])) if ((i.left != Allen.OPEN) and (i.right != Allen.OPEN)) else None, i.upper - i.lower )  for i in total_intervals]
    return r

def interval_structure_as_durations(intervals):
    return {tl: intervals_as_durations(intervals[intervals[T] == tl]) for tl in range(len(STATE_VARIABLES)) }

def compute_support(intervals):
    pattern = baseLexRepr(data_to_words(interval_structure_as_durations(intervals))).as_regex
    matches = 0
    for t in TRANSACTIONS:
        if re.search(pattern, t) is not None:
            matches += 1
    return    matches/len(TRANSACTIONS)